In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import pandas as pd
print('CUDA available' if tf.config.list_physical_devices('GPU') else 'CUDA not available')

CUDA available


**1. Загрузка данных**

In [3]:
#Загрузка модели InceptionResNetV2
from tensorflow.keras.applications import InceptionResNetV2

225209952/225209952 [==============================] - 21s 0us/step


In [4]:
import os
from random import shuffle
from glob import glob

IMG_SIZE = (224, 224)  # размер входного изображения сети

train_files = glob(r'D:\НЕТОЛОГИЯ\Comp Vision\файлы\dogs-vs-cats-redux-kernels-edition\train/*.jpg')
test_files = glob(r'D:\НЕТОЛОГИЯ\Comp Vision\файлы\dogs-vs-cats-redux-kernels-edition\test/*.jpg')

# загружаем входное изображение и предобрабатываем
def load_image(path, target_size=IMG_SIZE):
    img = plt.imread(path)[...,::-1]
    img = cv2.resize(img, target_size)
    return tf.keras.applications.inception_resnet_v2.preprocess_input(img)

# функция-генератор загрузки обучающих данных с диска
def fit_generator(files, batch_size=32):
    batch_size = min(batch_size, len(files))
    while True:
        shuffle(files)
        for k in range(len(files) // batch_size):
            i = k * batch_size
            j = i + batch_size
            if j > len(files):
                j = - j % len(files)
            x = np.array([load_image(path) for path in files[i:j]])
            y = np.array([1. if os.path.basename(path).startswith('dog') else 0.
                          for path in files[i:j]])
            yield (x, y)
# функция-генератор загрузки тестовых изображений с диска
def predict_generator(files):
    while True:
        for path in files:
            yield np.array([load_image(path)])

**2. Создание модели**

In [ ]:
iresnet_base = InceptionResNetV2(include_top = False,
    weights='imagenet',
    input_shape = (IMG_SIZE[0], IMG_SIZE[1], 3),
    classes = 2,
    classifier_activation='softmax')

In [19]:
# фиксируем все веса предобученной сети
for layer in iresnet_base.layers:
    layer.trainable = False

# Вместо 7 послдених слоев добавлем один полносвязный слой
x = iresnet_base.layers[-9].output
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(1, activation='sigmoid', kernel_regularizer=tf.keras.regularizers.l1(1e-4))(x)

model = tf.keras.Model(inputs = iresnet_base.input, outputs = x, name='dogs_vs_cats')

**3. Компиляция и обучение модели**

In [21]:
model.compile(optimizer='adam', 
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [24]:
val_samples = 5  # число изображений в валидационной выборке

shuffle(train_files)  # перемешиваем обучающую выборку
validation_data = next(fit_generator(train_files[:val_samples], val_samples))
train_data = fit_generator(train_files[val_samples:])  # данные читаем функцией-генератором

# запускаем процесс обучения
model.fit(train_data,
          steps_per_epoch = 10,  # число вызовов генератора за эпоху
          epochs = 20,  # число эпох обучения
          validation_data=validation_data)

Epoch 1/20
10/10 [==============================] - 22s 1s/step - loss: 0.7368 - accuracy: 0.6125 - val_loss: 0.2111 - val_accuracy: 1.0000
Epoch 2/20
10/10 [==============================] - 7s 720ms/step - loss: 0.3420 - accuracy: 0.8687 - val_loss: 0.1899 - val_accuracy: 1.0000
Epoch 3/20
10/10 [==============================] - 7s 728ms/step - loss: 0.2457 - accuracy: 0.9375 - val_loss: 0.1548 - val_accuracy: 1.0000
Epoch 4/20
10/10 [==============================] - 7s 756ms/step - loss: 0.2217 - accuracy: 0.9281 - val_loss: 0.0824 - val_accuracy: 1.0000
Epoch 5/20
10/10 [==============================] - 7s 744ms/step - loss: 0.1620 - accuracy: 0.9594 - val_loss: 0.0717 - val_accuracy: 1.0000
Epoch 6/20
10/10 [==============================] - 7s 720ms/step - loss: 0.1817 - accuracy: 0.9406 - val_loss: 0.0386 - val_accuracy: 1.0000
Epoch 7/20
10/10 [==============================] - 8s 830ms/step - loss: 0.1679 - accuracy: 0.9469 - val_loss: 0.0788 - val_accuracy: 1.0000
Epoch 8/

In [31]:
model.save('D:\НЕТОЛОГИЯ\Comp Vision\файлы\cats-dogs-inresnetv2.keras')

In [8]:
model = tf.keras.models.load_model('D:\НЕТОЛОГИЯ\Comp Vision\файлы\cats-dogs-inresnetv2.keras')

**4. Предсказания на тестовой выборке**

In [9]:
test_pred = model.predict(predict_generator(test_files), steps=len(test_files))

12500/12500 [==============================] - 406s 32ms/step


In [15]:
import re

with open(r'D:\НЕТОЛОГИЯ\Comp Vision\файлы\dc_prediction.csv', 'w') as dst:
    dst.write('id,label\n')
    for path, score in zip(test_files, test_pred):
        dst.write('%s,%f\n' % (re.search('(\d+).jpg$', path).group(1), score))